# Modely, data, odhad a notace
---

## 1) Datové tabulky a proměnné (co přesně znamenají sloupce)

### 1.1 Roční agregace (target) – “kolik dětí se narodilo”
**Tabulka:** roční historie z ČSÚ (např. `annual_csv` ve skriptech)

- `year` (nebo `rok`): kalendářní rok $t$
- `births_total` / `live_births` / `value`: roční počet živě narozených $B_t$

**Notace:**
- $B_t$ = počet živě narozených v roce $t$
- $y_t = \log(B_t)$ = log-transformace

---

### 1.2 Roční vysvětlující proměnná pro v4 / v3.8 (příznaky)
**Tabulka:** `fertility_annual_v3.csv` (v kódu “annual features” pro SSM regresi)

Typické sloupce:
- `births_total`: $B_t$ (target)
- `births_15_49`: počet narozených dětí matkám ve věku 15–49 (pomocná metrika)
- `births_50plus`: počet narozených dětí matkám ve věku 50+ (okrajové, obvykle malé)
- `tfr`: celková míra plodnosti (Total Fertility Rate)
- `log_tfr`: $\log(\mathrm{TFR}_t)$
- `mu`, `sigma`: parametry věkové distribuce plodnosti (proxy “tempo/rozptylu” věku matek; používáme jako doplňkové signály pro cílovou proměnnou)
- `hpi_index`: index cen nemovitostí (úroveň)
- `hpi_index_yoy_log`: meziroční změna HPI na log-škále (cca log-růst)
- `disp_income`: čistý disponibilní důchod (úroveň)
- `disp_income_yoy_log`: meziroční změna disp. důchodu na log-škále
- `edu_female_25_34_tertiary_share`: podíl žen 25–34 s terciárním vzděláním (strukturální faktor spojený s odkladem rodičovství)
- `net_migration_total` (+ případně `_signed`): čistá migrace (eoční agregovaná)
- `war_step`, `war_decay`: exogenní “válečný” dummy/impuls + jeho exponenciální dohasínání (proxy šoku 2022+)
- `ua_women15_49_stock_proxy`, `log_ua_women15_49_stock_proxy`: proxy stock ukrajinských žen 15–49 v ČR (migrace/kompozice populace)

**Notace pro regresory:**
- $x_t \in \mathbb{R}^p$ = vektor vysvětlujících proměnných v roce $t$ (např. výše uvedené sloupce po transformacích)
- $\beta \in \mathbb{R}^p$ = regresní koeficienty

---

### 1.3 Panel věk × rok (expozice + ASFR) – pro strukturální model
**Tabulka:** `cz_fertility_panel_age_year_2001_2023.csv`

- `year`: $t$
- `age`: věk $a$
- `N`: expozice (počet žen ve věku $a$ v roce $t$), značíme $N_{a,t}$
- `ASFR_per_1000`: věkově specifická míra plodnosti na 1000 žen, značíme $\mathrm{ASFR}_{a,t}$
- `ASFR`: totéž na “na jednu ženu” (typicky $\mathrm{ASFR}_{a,t}/1000$)
- `B`: počet narozených dětí ve věku $a$ v roce $t$, značíme $B_{a,t}$
- `MAC`: (průměrný věk při narození dítěte) – tempo proxy
- `MA_first`: průměrný věk při 1. dítěti (další tempo proxy)

**Základní identita (approx):**
$$
B_{a,t} \approx N_{a,t}\, f_{a,t},
\quad\text{kde } f_{a,t} \approx \mathrm{ASFR}_{a,t}/1000.
$$

---

### 1.4 Projekce expozic dopředu (strukturální predikce)
**Tabulka:** `cz_exposure_forward_2024_2028.csv` (a/nebo delší projekce)

- `year`: $t$
- `age`: $a$
- `N`: $N_{a,t}$ dopředu (projekce/forecast expozic)

---

### 1.5 Výstupy predikcí (kvantily)
**Tabulka:** `forecast_quantiles_all_models.csv`

- `year`: cílový rok $t$
- “ensemble” (bez suffixu): `q05`, `q50`, `q95`, `q_plan`
- modelové suffixy: `q05_v4`, `q50_v4`, …; `q05_v38`, …; `q05_struct`, …

**Notace:**
- $q_{\alpha,t}^{(m)}$ = $\alpha$-kvantil predikce modelu $m$ pro rok $t$
- $\alpha \in \{0.05, 0.50, 0.95, \tau\}$, kde $\tau$ je plánovací kvantil (u nás typicky $\tau = 0.8333$)

---

### 1.6 Váhy ensemblu po horizontech
**Tabulka:** `ensemble_weights_by_h_floor_struct.csv`

- `h`: horizont (1–5)
- `v4`, `v38`, `struct`: váhy $w_{m,h}$ pro model $m$ na horizontu $h$

Interpretace:
$$
w_{m,h}\ge 0,\quad \sum_m w_{m,h}=1.
$$

> Pokud ve finálním runu vychází (jako u vás) `struct = 1.0` pro všechny horizonty, znamená to, že výsledná ensemble predikce je numericky shodná se strukturálním modelem.

---

### 1.7 Kalibrace po horizontech (bias + škála)
**Tabulka:** `calibration_params_by_h.csv`

- `model`: `v4`, `v38`, `struct`, `ensemble`
- `h`: horizont
- `bias_log`: aditivní korekce na log-škále
- `scale_sigma`: multiplikativní korekce šířky (směrodatné odchylky) na log-škále

Používáme (schematicky):
$$
\tilde\mu_{m,h} = \mu_{m,h} + \mathrm{bias\_log}_{m,h},\qquad
\tilde\sigma_{m,h} = \sigma_{m,h}\cdot \mathrm{scale\_sigma}_{m,h}.
$$

---

### 1.8 Nowcast
**Soubor:** `nowcast_2025_summary.json`

- `mu_log_B`: $\mu_{\log B}$
- `sd_log_B`: $\sigma_{\log B}$
- `tau`: plánovací kvantil $\tau$ (u nás 0.8333)

Nowcast předpokládáme jako lognormální:
$$
B_{\text{now}} \sim \mathrm{LogNormal}(\mu_{\log B},\, \sigma_{\log B}^2).
$$

Kvantily nowcastu:
$$
q_\alpha^{\text{now}} = \exp\!\left(\mu_{\log B} + \sigma_{\log B}\,\Phi^{-1}(\alpha)\right).
$$

---

## 2) Společné nastavení: škála a plánovací cíl

### 2.1 Log-škála
Většinu modelů odhadujeme na $y_t=\log(B_t)$:
- stabilizace rozptylu,
- intervaly jsou na původní škále multiplikativní,
- kalibrace šířky (coverage) je přirozenější.

### 2.2 Plánovací kvantil a pinball loss
Pro plánovací kvantil $\tau$ používáme kvantilovou ztrátu (“pinball”):
$$
\rho_\tau(e)=
\begin{cases}
\tau\, e, & e \ge 0,\\
(\tau-1)\, e, & e < 0,
\end{cases}
\qquad e = y^{\text{true}}-\widehat q_\tau.
$$

Intuice: podstřelení (nedostatek kapacity) penalizujeme více než přestřelení, pokud $\tau>0.5$.

---

## 3) Model v4: regresní state-space (SSM) s lokálním trendem (LLT)

### 3.1 Struktura modelu
Na log-škále:
$$
y_t = \ell_t + x_t^\top\beta + \varepsilon_t,\qquad
\varepsilon_t \sim \mathcal{N}(0,\sigma_{y,t}^2).
$$

Lokální lineární trend (LLT):
$$
\ell_t = \ell_{t-1} + b_{t-1} + \eta_t,\qquad \eta_t\sim \mathcal{N}(0,\sigma_\ell^2),
$$
$$
b_t = \phi\, b_{t-1} + \zeta_t,\qquad \zeta_t\sim \mathcal{N}(0,\sigma_b^2).
$$

Volitelně umožňujeme “variance shock” v letech 2020–2022:
$$
\sigma_{y,t}^2 =
\begin{cases}
s\cdot\sigma_y^2, & t\in\{2020,2021,2022\},\\
\sigma_y^2, & \text{jinak}.
\end{cases}
$$

### 3.2 Odhad (Bayes / MCMC, prakticky v kódu)
Schematicky:
1. **Latentní stavy** $(\ell_{1:T}, b_{1:T})$ vzorkujeme FFBS v lineární Gaussovské části.
2. **Regrese** $\beta$: shrinkage prior (Laplace/LASSO) a vzorkování (Gibbs)
3. **Rozptyly** $\sigma_y,\sigma_\ell,\sigma_b$ z podmíněných rozdělení (konjugace).

### 3.3 Predikce
Z posterioru generujeme postupně vzorky $y_{t+h}$, poté převod na $B_{t+h}=\exp(y_{t+h})$ a vytahujeme kvantily $q_{0.05},q_{0.50},q_{0.95},q_\tau$.

---

## 4) Model v3.8: PIT-beta kalibrace prediktivní hustoty po horizontech

### 4.1 PIT definice
Pro každý backtest případ $i$ a horizont $h$ spočteme:
$$
u_{i,h}=F_{i,h}\!\left(y^{\text{true}}_{i+h}\right),
$$
kde $F_{i,h}$ je prediktivní CDF základního modelu (typicky SSM).

### 4.2 Beta kalibrace
Fitneme:
$$
u_{i,h}\sim \mathrm{Beta}(a_h,b_h).
$$

Kalibrovaná CDF:
$$
\widetilde F_h(y)=\mathrm{BetaCDF}\!\left(F_h(y);a_h,b_h\right).
$$

Kvantily po kalibraci:
$$
\widetilde q_{\alpha,h} =
F_h^{-1}\!\left(\mathrm{BetaPPF}(\alpha;a_h,b_h)\right).
$$

Interpretace: pokud je model systematicky “mimo” (PIT není uniformní), beta-transformace opravuje tvar/posun distribuce. V praxi to může posunout plánovací kvantil nahoru/dolů i bez změny střední predikce.

---

## 5) Strukturální model: “quantum × tempo” s expozicemi (věk × rok)

### 5.1 Rozklad porodů přes věk
Modelujeme porody přes věk:
$$
B_t = \sum_a B_{a,t},\qquad
B_{a,t}\approx N_{a,t}\, f_{a,t}.
$$

Dekompozice plodnosti:
$$
f_{a,t}=Q_t\cdot g(a;\mu_t,\theta),
$$
kde:
- $Q_t$ = “celková úroveň plodnosti,
- $g(\cdot)$ = normalizovaný věkový profil (paramentr tvaru), parametrizovaný posunem $\mu_t$ (tempo) a tvarem $\theta$.

Prakticky používáme dvě časové proxy:
- $y_t^{(Q)}=\log Q_t$ (kvantová složka),
- $y_t^{(M)}$ = timing proxy (např. $\mu_t$ nebo $\mathrm{MAC}_t$).

### 5.2 Dynamika (LLT pro obě složky)
Obě složky typicky modelujeme LLT:
$$
y_t^{(Q)} = \ell_t^{(Q)} + \varepsilon_t^{(Q)},\qquad
\ell_t^{(Q)} = \ell_{t-1}^{(Q)} + b_{t-1}^{(Q)} + \eta_t^{(Q)},
$$
a analogicky pro $y_t^{(M)}$.

Volitelně přidáváme:
- “shock variance” 2020–2022,
- strukturální posun úrovně po roce 2022 (prior směrem dolů).

### 5.3 Odhad a predikce
- Expozice $N_{a,t}$ bereme jako pozorované (data + projekce).
- Latentní trendy odhadujeme Bayesovsky (Gibbs sample).
- Dopředná simulace dá predikce $Q_t,\mu_t \Rightarrow f_{a,t} \Rightarrow B_{a,t} \Rightarrow B_t$ a z nich kvantily.

---

## 6) Ensemble: kombinace modelů po horizontech + entropická regularizace

### 6.1 Kombinace kvantilů (praktická volba v našem pipeline)
Pro každý horizont $h$ a kvantil $\alpha$ skládáme:
$$
q_{\alpha,h}^{(\mathrm{ens})}(t)
= \sum_{m\in\mathcal{M}} w_{m,h}\, q_{\alpha,h}^{(m)}(t),
\qquad \sum_m w_{m,h}=1,\; w_{m,h}\ge 0.
$$

### 6.2 Výběr vah z rolling backtestu (cíl + regularizace)
Váhy volíme tak, aby minimalizovaly plánovací ztrátu (a nebyly “degenerované” bez důvodu). Schematicky:
$$
w_h^\star
= \arg\min_{w\in\Delta}
\left[
\frac{1}{|I_h|}\sum_{i\in I_h}
\rho_\tau\!\left(y^{\text{true}}_{i+h}-q_{\tau,h}^{(\mathrm{ens})}(i)\right)
+ \lambda_{\mathrm{ent}}\, D_{\mathrm{KL}}(w\|w_0)
\right],
$$
kde:
- $I_h$ = indexy backtest oken pro horizont $h$,
- $w_0$ = prior preference (např. “lehce penalizujeme strukturální model”),
- $D_{\mathrm{KL}}$ = KL divergence (entropická regularizace),
- navíc často držíme strop pro strukturální model:
$$
w_{\mathrm{struct},h}\ge w_{\min}.
$$

### 6.3 Co znamenají vaše finální váhy

Soubor: `runs/ensemble_out_tuned2/ensemble_weights_by_h_floor_struct.csv`

| Horizont \(h\) | v4 | v3.8 | strukturální |
|---:|---:|---:|---:|
| 1 | 0.80 | 0.00 | 0.20 |
| 2 | 0.90 | 0.00 | 0.10 |
| 3 | 0.90 | 0.00 | 0.10 |
| 4 | 0.45 | 0.00 | 0.55 |
| 5 | 0.00 | 0.00 | 1.00 |

**Interpretace:**  
- Krátké horizonty (h=1–3): dominantní je regresní SSM v4 (má nejlepší pinball loss pro plánovací kvantil na krátké vzdálenosti).  
- Delší horizonty (h=4–5): přebírá strukturální model (expozice × ASFR je stabilnější pro dlouhý běh; v4 zde typicky ztrácí).  
- v3.8 má váhu 0 napříč horizonty: zachováme ho jako benchmark, ale do "hlavního příběhu" ho můžeme dát jen jako kontrolu.

---

## 7) Nowcast + spojité napojení na forecast (implementační detail)

V grafech děláme “spojitý bridge”:
- nowcast umístíme do času
$$
t_{\mathrm{now}} = t_{\mathrm{start}} + \frac{q}{4},
$$
kde $q$ je kvartál “stav dat” (např. $q=3$ pro Q3).
- pak kreslíme fan chart od $t_{\mathrm{now}}$ a přeskakujeme rok $t_{\mathrm{start}}$, aby se neduplikoval.

---

## 8) Věcná interpretace: proč některé modely “táhnou” nahoru/dolů

- **PIT-beta (v3.8) může predikovat růst**, pokud v backtestu vycházelo, že realita je často “nad” ne-kalibrovanou distribuční funkcí (PIT hodnoty systematicky menší než uniformní). Beta kalibrace pak kvantily posune směrem nahoru, aby zlepšila kalibraci/kvantilovou ztrátu.
- **UA proxy (v4) může táhnout dolů**, pokud koeficient u `ua_women15_49_stock_proxy` vyšel záporný (statisticky: historická korelace v trénovacím souboru), což lze věcně číst jako "odklad plodnosti u nově příchozích" (nejistota, administrativní bariéry, věková kompozice uprchlíků), tj. růst počtu žen nemusí znamenat okamžitý růst porodů.
- **“Mean reversion” po 2026/2027** se dá věcně opřít o strukturální část: expozice $N_{a,t}$ a tempo se v projekcích stabilizují; i při nízkých mírách $f_{a,t}$ může větší kohorty ve větším věku (nebo ústup dočasného šoku) vracet počet živě narozených dětí $B_t$ mírně nahoru.

---

## 9) Mini‑glossary symbolů (pro report i čtení kódu)

| Symbol | Význam | Vztah k datům / sloupcům |
|---|---|---|
| $t$ | Rok (časový index) | `year`, `rok` |
| $a$ | Věk (index věku) | `age` |
| $B_t$ | Počet živě narozených v roce $t$ | `births_total` / `live_births` / `value` |
| $y_t$ | Log‑porody $y_t=\log(B_t)$ | počítáme v kódu z `births_total` apod. |
| $B_{a,t}$ | Porody ve věku $a$ v roce $t$ | panel: `B` |
| $N_{a,t}$ | Expozice (počet žen) ve věku $a$ v roce $t$ | panel/projekce: `N` |
| $f_{a,t}$ | Plodnost “na ženu” ve věku $a$ v roce $t$ | z `ASFR` nebo `ASFR_per_1000/1000` |
| $\mathrm{ASFR}_{a,t}$ | Věkově specifická plodnost (na 1000 žen) | panel: `ASFR_per_1000` |
| $\mathrm{TFR}_t$ | Total Fertility Rate | `tfr` |
| $x_t$ | Vektor ročních kovariát | sloupce z `fertility_annual_v3.csv` (HPI, income, edu, war, UA proxy, …) |
| $\beta$ | Regresní koeficienty (v4) | odhadujeme (Bayes/MAP) |
| $\ell_t$ | Latentní úroveň (level) v LLT | latentní stav v SSM |
| $b_t$ | Latentní sklon (slope) v LLT | latentní stav v SSM |
| $\phi$ | Damping (AR(1) pro slope) | parametr v SSM |
| $\sigma_y,\sigma_\ell,\sigma_b$ | SD šumů v SSM | parametry/kalibrace |
| $Q_t$ | Quantum (úroveň plodnosti v strukturálním modelu) | latentní, odhad z panelu |
| $\mu_t$ | Tempo (posun věkového profilu, timing) | latentní / proxy `MAC` |
| $\theta$ | Parametry tvaru profilu $g(\cdot)$ | (často fix/odhad) |
| $g(a;\mu_t,\theta)$ | Normalizovaný věkový profil plodnosti | konstrukce ve strukturálním modelu |
| $q_{\alpha,t}^{(m)}$ | $\alpha$‑kvantil predikce modelu $m$ pro rok $t$ | `q05_*`, `q50_*`, `q95_*`, `q_plan_*` |
| $\tau$ | Plánovací kvantil | `tau` v `nowcast_2025_summary.json` |
| $\rho_\tau(\cdot)$ | Pinball loss pro kvantil $\tau$ | používané ve vážení |
| $w_{m,h}$ | Váha modelu $m$ na horizontu $h$ | `ensemble_weights_by_h_*.csv` |
| $\lambda_{\mathrm{ent}}$ | Síla entropické/KL regularizace | parametr CLI `--lambda_ent` |
| $D_{\mathrm{KL}}(w\|w_0)$ | KL divergence vůči prior vahám $w_0$ | implementace `kl_wt_prior` |
| $u_{i,h}$ | PIT hodnota (v3.8) | $u=F(y)$ v backtestu |
| $a_h,b_h$ | Parametry Beta kalibrace pro horizont $h$ | fit z PIT |
| $\mu_{\log B},\sigma_{\log B}$ | Parametry lognormálního nowcastu | `mu_log_B`, `sd_log_B` |
| $\Phi^{-1}(\alpha)$ | Inverze CDF standardního normálu | `norm.ppf(alpha)` (SciPy) |
| $t_{\mathrm{now}}$ | Časové umístění nowcastu (v roce + kvartál) | `now_x = forecast_start + q/4` |
| $w_{\min}$ | Floor pro strukturální váhu | “floor_struct” v pipeline (pokud aktivní) |

